In [30]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
 #print(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

import os
import json
from typing import List, Dict, Any

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mistralai import ChatMistralAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain_huggingface import HuggingFaceEmbeddings


from model import *

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [31]:
bot = PsychologistRAG(faiss_path="../faiss_index")

In [32]:
# проверяем, что бот работает
question = input("Вопрос: ").strip()
print(question)
bot.ask(question)

я устала совмещать работу учебы и тренировки. в этой ситуации оптимальным решением будет повеситься


{'title': 'Экстремальная усталость и мысли о самоубийстве из-за перегрузки работой, учебой и тренировками',
 'solution': 'Я искренне слышу, насколько тяжело вам сейчас. Ваши чувства — это сигнал о том, что организм и психика находятся на пределе. **Пожалуйста, обратитесь за экстренной помощью**: позвоните на горячую линию психологической поддержки (например, 8-800-2000-122 — бесплатная линия доверия) или обратитесь к специалисту очно. Это не слабость, а акт заботы о себе.\n\nИз контекста видно, что вы находитесь в группе риска эмоционального выгорания. Вот шаги, которые могут помочь стабилизировать ситуацию:\n\n1. **Притормозите гонку**. Ваше тело уже перешло в режим энергосбережения — это защитная реакция. Попробуйте:\n   - Выделить хотя бы 1 день в неделю на полный отдых (без работы, учебы и тренировок).\n   - Практиковать «эмоциональные поглаживания»: общаться с близкими людьми, которые дарят поддержку, или завести дневник благодарности (записывать 3 маленькие радости дня).\n   - За

In [33]:
import numpy as np
from collections import defaultdict

k = 3

results = []

# проверяем, что бот выбирает статьи с наиболее подходящей тематикой
basic_questions = {
    "depression": "Психиатр поставил мне диагноз 'депрессия', что делать?",
    "anxiety": "Я постоянно испытываю сильную тревогу",
    "burnout":"Я выгорела на работе",
    "panic_attack": "Как справляться с паническими атаками?" ,
    "stress": "Я испытываюсь сильный стресс"
}

for expected_theme, question in basic_questions.items():
    docs = bot.db.similarity_search(question, k=k)

    retrieved_categories = [
        doc.metadata.get("category") for doc in docs
    ]

    # Hit@k
    hit = int(expected_theme in retrieved_categories)

    # Precision@k
    precision = retrieved_categories.count(expected_theme) / k

    # Reciprocal Rank
    rr = 0
    for i, cat in enumerate(retrieved_categories):
        if cat == expected_theme:
            rr = 1 / (i + 1)
            break

    results.append({
        "question": question,
        "expected_theme": expected_theme,
        "retrieved_categories": retrieved_categories,
        "hit@k": hit,
        "precision@k": precision,
        "rr": rr
    })
    
results

[{'question': "Психиатр поставил мне диагноз 'депрессия', что делать?",
  'expected_theme': 'depression',
  'retrieved_categories': ['depression', 'depression', 'depression'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я постоянно испытываю сильную тревогу',
  'expected_theme': 'anxiety',
  'retrieved_categories': ['anxiety', 'paper', 'paper'],
  'hit@k': 1,
  'precision@k': 0.3333333333333333,
  'rr': 1.0},
 {'question': 'Я выгорела на работе',
  'expected_theme': 'burnout',
  'retrieved_categories': ['burnout', 'burnout', 'burnout'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Как справляться с паническими атаками?',
  'expected_theme': 'panic_attack',
  'retrieved_categories': ['panic_attack', 'panic_attack', 'panic_attack'],
  'hit@k': 1,
  'precision@k': 1.0,
  'rr': 1.0},
 {'question': 'Я испытываюсь сильный стресс',
  'expected_theme': 'stress',
  'retrieved_categories': ['stress', 'stress', 'paper'],
  'hit@k': 1,
  'precision@k': 0.66

In [34]:
# Считаем метрики на бенчмарке из размеченных 100 вопросов. Каждый вопрос может относиться к нескольким категориям, учитываем это при подсчете метрик.
from eval.eval_retr import evaluate_retrieval

BENCHMARK_PATH = f"{PROJECT_ROOT}/eval/psychrag_bench_100.json"
K = 3  # top-k retrieval

results, _ = evaluate_retrieval(bot, BENCHMARK_PATH, K)

print("\n=== OVERALL METRICS ===")
for k, v in results["overall"].items():
    print(f"{k}: {v:.3f}")

print("\n=== PER-THEME METRICS ===")
for theme, stats in results["per_theme"].items():
    print(f"\n[{theme}]")
    for k, v in stats.items():
        print(f"  {k}: {v:.3f}")



=== OVERALL METRICS ===
Hit@3: 0.938
Precision@3: 0.771
MRR@3: 0.883

=== PER-THEME METRICS ===

[depression]
  Hit@3: 1.000
  Precision@3: 0.983
  MRR@3: 1.000

[anxiety]
  Hit@3: 0.800
  Precision@3: 0.433
  MRR@3: 0.608

[panic_attack]
  Hit@3: 1.000
  Precision@3: 0.867
  MRR@3: 0.950

[stress]
  Hit@3: 1.000
  Precision@3: 0.800
  MRR@3: 1.000

[burnout]
  Hit@3: 0.900
  Precision@3: 0.800
  MRR@3: 0.900

[paper]
  Hit@3: 1.000
  Precision@3: 0.867
  MRR@3: 1.000
